# 基于医学规则的排牙与分步

该Notebook将演示：

1. 如何调用朝厚云接口生成自动加工单并根据自动加工单生成符合要求的排牙
2. 如何根据医生手动修改或编写的加工单自动生成符合加工单要求的排牙
3. 如何根据排牙生成分步方案

使用本Notebook需要准备一个病例的：
1. 侧位片
2. 微笑照
3. 上下颌口扫模型

In [ ]:
# 导入必要的包以及定义函数
import os
import time
import requests
import json
import trimesh
import urllib
import numpy as np

# 定义调用规则

请根据您从我方获取的信息修改以下代码块

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def retrieve_data(urn):
    return requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

## 自动加工单与目标位

该章节多个调用可以使用以下单一调用代替

```python
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-arrangement-medical", 
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id
}
```

### 获取分牙结果

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-denoise-prod",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"drc", "data": upload_file("upper_jaw_scan.drc")},
      "jaw_type": "Upper"
  },
  'output_config': {
    "teeth_comp": {"type": "ply"}
  }
}
result_upper_jaw = run_job_and_get_results(json_call, 300)

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-denoise-prod",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"ply", "data": upload_file("lower_jaw_scan.ply")},
      "jaw_type": "Lower"
  },
  'output_config': {
    "teeth_comp": {"type": "ply"}
  }
}
result_lower_jaw = run_job_and_get_results(json_call, 300)

### 获取侧位片与微笑像分析结果

In [ ]:
json_call = {
  "spec_group": "ceph",
  "spec_name": "ceph-analysis",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "image": upload_file("ceph.jpg")
  }
}
result_ceph = run_job_and_get_results(json_call, 20)

In [ ]:
json_call = {
  "spec_group": "face",
  "spec_name": "smile-analysis",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "image": upload_file("face_smile.jpg")
  }
}
result_smile = run_job_and_get_results(json_call, 20)

### 获取自动工单

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "auto-form",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "upper_teeth_dict": result_upper_jaw["teeth_comp"],
      "upper_align_matrix": result_upper_jaw["align_matrix"],
      "upper_axis_matrix_dict": result_upper_jaw["axis"],
      "lower_teeth_dict": result_lower_jaw["teeth_comp"],
      "lower_align_matrix": result_lower_jaw["align_matrix"],
      "lower_axis_matrix_dict": result_lower_jaw["axis"],
      "ceph_metric_pts_dict": result_ceph['result']["kps"],
      "frontal_smiling_pts_dict": result_smile['result']['kps'],
      "meta": result_ceph['result']['meta']
  }
}
result_form = run_job_and_get_results(json_call, 500)

In [ ]:
def form2str(form):
    def collision2str(collisions):
        collision_name_dict = {
            'left_back_teeth_IPR': '左侧后牙IPR',
            'front_teeth_IPR': '前牙IPR',
            'right_back_teeth_IPR': '右侧后牙IPR',
            'move_left_molar': '远移左侧磨牙',
            'move_right_molar': '远移右侧磨牙',
            'move_both_molar_forward': '磨牙近移'
        }

        return '，'.join(
            [collision_name_dict[
                 collision[0] if isinstance(collision, tuple) or isinstance(collision, list) else collision] for
             collision in
             collisions])

    cusp_name_dict = {
        'near_cusp': '近中切角',
        'mid_cusp': '切端',
        'far_cusp': '远中切角'
    }
    direction_name_dict = {
        'backward': '内收',
        'forward': '唇倾',
        'down': '压低',
        'up': '伸长',
        'left': '左移',
        'right': '右移'
    }

    form_str = f"""
    下颌目标位
    1. 前牙矢状向目标位：以__号牙__为准__毫米
       {form['front_y_axis_position'][0]}，{cusp_name_dict[form['front_y_axis_position'][1]]}，{direction_name_dict[form['front_y_axis_position'][2]]}{form['front_y_axis_position'][3]:.1f}
    2. 垂直向目标位：后牙不动，前牙__毫米
       {direction_name_dict[form['z_axis_position']['front_teeth'][0]]}{form['z_axis_position']['front_teeth'][1]:.1f}

    上下颌咬合关系
    1. 咬合关系【左侧】：尖牙到__类关系，磨牙到__类关系
       一，一
    2. 咬合关系【右侧】：尖牙到__类关系，磨牙到__类关系
       一，一
    3. 咬合关系【覆盖】：__覆盖
       标准
    4. 咬合关系【覆颌】：__覆颌，__后牙
       标准，维持
    5. 咬合关系【后牙反颌|锁颌】
       纠正

    中线目标位
    1. [上颌] 中线目标位
       {direction_name_dict[form['middle_line_position']['U'][1]]}{form['middle_line_position']['U'][2]:.1f}毫米
    2. [下颌] 中线目标位
       对齐上颌中线

    碰撞解除|间隙闭合
    1. 上颌
       {collision2str(form['collision_removal']['U']['1'])}
    2. 下颌
       {collision2str(form['collision_removal']['L']['1'])}
    3. 拔牙牙号
       {'，'.join(map(str, form['remove_teeth_set']))}
    4. 预留间隙
       {'，'.join([f'{k}={v}' for k, v in form['init_gap'].items()]) if len(form['init_gap']) > 0 else '无'}
    """

    return form_str

print(form2str(json.loads(result_form['result']['form'])))

### 根据工单排牙

使用自动工单格式的工单进行排牙（此工单也可以手动填写）

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "arrange-with-form",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "upper_teeth_dict": result_upper_jaw["teeth_comp"],
      "upper_align_matrix": result_upper_jaw["align_matrix"],
      "upper_axis_matrix_dict": result_upper_jaw["axis"],
      "lower_teeth_dict": result_lower_jaw["teeth_comp"],
      "lower_align_matrix": result_lower_jaw["align_matrix"],
      "lower_axis_matrix_dict": result_lower_jaw["axis"],
      "form": result_form['result']['form'],
      "matrix_3d": result_form['result']['matrix_3d'],
  },
  'output_config': {
    "arranged_comp": {"type": "ply"}
  }
}
result_arrangement = run_job_and_get_results(json_call, 500)

In [ ]:
sum([retrieve_mesh(result_arrangement['result']['arranged_comp'][k]) for k in result_arrangement['result']['arranged_comp'].keys()]).show()

## 自动分步方案


In [ ]:
json_call = {
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "auto-step", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}
json_call["input_data"] = {
    "upper_teeth_dict": result_upper_jaw["teeth_comp"],
    "upper_align_matrix": result_upper_jaw["align_matrix"],
    "upper_axis_matrix_dict": result_upper_jaw["axis"],
    "lower_teeth_dict": result_lower_jaw["teeth_comp"],
    "lower_align_matrix": result_lower_jaw["align_matrix"],
    "lower_axis_matrix_dict": result_lower_jaw["axis"],
    "transformation_dict": result_arrangement["result"]['transformation_dict']
}
result_step = run_job_and_get_results(json_call, 2200)

In [ ]:
print("step count:", len(result_step['result']['step_dict']))

In [ ]:
original_comp = {**{k: retrieve_mesh(v) for k, v in result_upper_jaw["teeth_comp"].items()},
                 **{k: retrieve_mesh(v) for k, v in result_lower_jaw["teeth_comp"].items()}}
def show_step(step_index):
    result_mesh = None
    for k, m in original_comp.items():
        if k in result_step['result']['step_dict'][step_index]:
            result_mesh += m.copy().apply_transform(result_step['result']['step_dict'][step_index][k])
    return result_mesh

In [ ]:
# 第10步展示
show_step(10).show()